# 目标检测 Object Detection

## 1. 目标定位 Object Localization

目标检测是这几年随着深度学习兴起以来，计算机视觉领域中进展神速的一个问题。要进行目标检测，目标定位是第一步。

目标定位就是在一个图像中，找到目标，用一个包围盒将其标出，目标定位通常和分类器配合使用，只涉及一个目标。而目标检测则在此基础上，进行多个目标的标记。

![What are localization and detection.png](img/What are localization and detection.png)

带目标定位的神经网络分类器，就是在卷积神经网络分类器的基础上，在最后的Softmax层，加入红框的坐标信息。这里记做 $b_x, b_y, b_h, b_w$，分别表示包围盒中心的横坐标，包围盒中心的纵坐标，包围盒的高度和包围盒的宽度。

![Classification with localization.png](img/Classification with localization.png)

最终输出的预测值 $\hat{y}$，包括三个部分：$p_c$ 图像中是否包含目标，$b_x, b_y, b_h, b_w$ 包围盒坐标信息，以及 $c_1, c_2, c_3$ 分类器分类信息。当图像中不包含目标时，包围盒坐标信息和分类信息实际上是我们不关心的，所以损失函数也会相应地调整，在 $y=0$ 时，只计算 $p_c$ 的误差。下面的损失函数以误差平方为例，考虑到是否包含目标以及分类信息都属于分类问题，这两部分的损失也可以用logloss来计算。

![Defining the target label y.png](img/Defining the target label y.png)

## 2. 特征点检测 Landmark Detection

类似上面带目标定位的神经网络分类器，也可以让神经网络最终的输出层直接输出**特征点 Landmark**的坐标。但特征点的数量需要预先定义。并且每个特征点在输入向量中的位置（比如左眼内侧眼角，下巴，膝盖），需要跨样本保持一致。

![Landmark detection.png](img/Landmark detection.png)

## 3. 滑动窗口目标检测 Sliding Windows Object Detection

使用滑动窗口目标检测算法，需要首先训练一个分类器。以汽车检测为例，这个分类器要求输入 $X$ 在正样本的情况下，图片的大小几乎只包含汽车。分类器本身，可以是神经网络，也可以不是。

![Car detection example.png](img/Car detection example.png)

滑动窗口目标检测的算法，先挑选一个较小的滑动窗口，配以一个最好较小的步长，在新的训练集图片中进行滑动，并将滑窗里截取的图片作为输入，用到上面训练好的分类器中。一个循环结束后，再放大滑动窗口的大小，继续之前的步骤。

滑动窗口的一个问题是计算成本较大，增加步长可以降低计算成本，但会损失精度。在神经网络兴起之前，滑动窗口目标检测，通常配合着人工特征以及简单线性分类器，计算量较小。而在深度学习时代，简单地用滑动窗口配合卷积神经网络，即使以当前的计算能力，还是非常地慢。

![Sliding windows detection.png](img/Sliding windows detection.png)

下一节，我们会介绍用卷积来实现滑动窗口，从而将计算量降低到可以接受的范围。

## 4. 滑动窗口的卷积实现 Convolutional Implementation of Sliding Windows

在介绍滑动窗口的卷积实现之前，我们先介绍一个概念，全连接（FC）层，是可以等价地用卷积层来表达的，如下图所示：

![Turning FC layer into convolutional layers.png](img/Turning FC layer into convolutional layers.png)

假设我们已经训练好了一个接受14×14×3的图片的卷积神经网络分类器。那么，针对更大的训练样本，我们只需要复制卷积神经网络的架构，就可以同步对多个滑动窗口的结果进行分类。

![Convolution implementation of sliding windows.png](img/Convolution implementation of sliding windows.png)

## 5. 预测包围盒 Bounding Box Prediction

上一节提到滑动窗口的卷积实现，计算上效率很高，但仍然有一个问题，它无法有效地确定包围盒。要使用不同尺寸的包围盒，需要训练多个不同的分类器。另外，实际训练样本的包围盒，其形状可能不是正方形，同时各个样本之间还存在差异。**YOLO(You Only Look Once)**算法，提供了更精确的包围盒预测算法。

在YOLO算法中，每张图片被划分为若干个区域，图上是3×3，实际中一般更大（比如19×19）。每个目标，根据其中心点，只被分配到一个具体的区域中。每个区域的目标变量y等同于目标定位中的目标变量。这样神经网络会输出图像中每个区域的包围盒（如果有的话）的精确位置。一个区域中有多个目标的情况，我们之后会再谈到。

YOLO算法是通过卷积实现的，因而效率非常高，经常用于实时的目标检测。

![YOLO algorithm.png](img/YOLO algorithm.png)

注意包围盒参数的取值范围。按照图像问题的惯例，图像的左上角定义为坐标(0, 0)，而右下角定义为坐标(1, 1)，$b_x$ 和 $b_y$ 的值根据这个坐标系来确定，由于是中心点的坐标，因此这两个值都在(0, 1)的区间内。但是目标是可以跨区域的，因而 $b_h$ 和 $b_w$ 是可以大于1的。

![Specify the bounding boxes.png](img/Specify the bounding boxes.png)

## 6. 交并集 Intersection Over Union

交并集可以作为评价计算出包围盒好坏的一个指标。在下一节，也会作为一个组件，提升目标检测算法。

计算机视觉领域惯例上在 $IoU \ge 0.5$ 时，判断包围盒正确。0.5在这里是一个人为的规定，并没有理论原因，如果希望判定更为严格，也可以将这个阈值设为0.6。

![Evaluating object localization.png](img/Evaluating object localization.png)

## 7. 非极大值抑制 Non-Max Suppression

YOLO算法针对同一个目标，可能会有多个区域判定该目标在本区域内，从而对单个目标产生多个包围盒。

![Non-max suppression example.png](img/Non-max suppression example.png)

利用IoU，可以消除针对同一目标的多个包围盒。这里就是挑选概率最高的包围盒，然后如果 $IoU \ge 0.5$，则认为是针对同一目标，直接消除。

![Non-max suppression algorithm.png](img/Non-max suppression algorithm.png)

## 8. Archor Box

目前为止上面提到的目标识别算法，还无法针对同一区域包含多个目标的情况进行识别。引入Archor Box的概念，可以解决这个问题。

![Overlapping objects.png](img/Overlapping objects.png)

![Archor box algorithm.png](img/Archor box algorithm.png)

![Archor box example.png](img/Archor box example.png)

Archor Box无法很好地处理：
    - 预先定义了两个Archor Box，但是区域中有三个目标的情况
    - 两个目标的Archor Box形状非常类似的情况

## 9. YOLO算法

![YOLO Training.png](img/YOLO Training.png)

![YOLO Making Predictions.png](img/YOLO Making Predictions.png)

![YOLO Outputing the non- max supressed outputs.png](img/YOLO Outputing the non- max supressed outputs.png)

## 10. Region Proposal

通过传统的图像分割算法，分割出区域，针对具体的区域进行分类。

![Region proposal R-CNN.png](img/Region proposal R-CNN.png)

这个算法比较慢，社区内一直在致力于提升其运算效率。

![Faster algorithms.png](img/Faster algorithms.png)

## 11. 车辆检测 Autonomous driving - Car detection

本节我们将会使用YOLO模型进行目标检测，YOLO算法主要来自于以下两篇论文：Redmon et al., 2016 (https://arxiv.org/abs/1506.02640) and Redmon and Farhadi, 2016 (https://arxiv.org/abs/1612.08242). 

**这一节，我们将会**:
- 对一个车辆检测数据集，应用目标检测算法
- 处理包围盒

执行以下代码来引入相关的包和依赖。

In [1]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes
from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body

%matplotlib inline

Using TensorFlow backend.


### 11.1 问题陈述

作为研发自动驾驶汽车的一个核心组件，我们需要首先构建一个车辆检测系统。为了收集数据，我们在车辆的前方挂载了一个摄像头，行驶过程中每隔几秒就对路面情况进行拍照。

<center>
<video width="400" height="200" src="img/road_video_compressed2.mp4" type="video/mp4" controls>
</video>
</center>

所有的图片现在都放到了一个文件夹中，并且已经完成了打标，每张图片上，每辆车都画出了一个包围盒。下面是一个包围盒的实例。

<img src="img/box_label.png" style="width:500px;height:250;">
<caption><center> <u> **Figure 1** </u>: **Definition of a box**<br> </center></caption>

如果我们目前有80个分类需要YOLO算法来识别，可以将类别标签 $c$ 表示为一个1到80的整型数字，或者一个80维的向量（其中一个元素是1，其它都是0）。本节中，我们会视每一步的使用便利，来决定使用哪一种表示。

这个练习中，我们会学得YOLO是如果运作的，之后再将其应用于车辆检测。由于YOLO模型训练的过程计算消耗很大，我们会载入一个预先训练好的模型。

### 11.2 YOLO

YOLO（只看一次）是目前非常流行的目标检测算法，它的准确率高，同时还能够实时进行计算。只看一次意味着只需要一次前向传播，算法就可以做出预测。在进行了非极大值抑制之后，算法就可以产出识别的目标以及对应的包围盒。

#### 11.2.1 模型细节

数据情况：
- **输入input** 是一批图片，维度为(m, 608, 608, 3)
- **输出output** 是一组包围盒及其对应的分类名称。每个包围盒由6个数字 $(p_c, b_x, b_y, b_h, b_w, c)$ 来表示，如上所述。如果将 $c$ 扩展为一个80维的向量，那么每个包围盒就可以表示为85个数字。

我们将会使用5个archor boxes。所以这个YOLO的架构可以认为是：IMAGE (m, 608, 608, 3) -> DEEP CNN -> ENCODING (m, 19, 19, 5, 85)，encoding的部分如图所示：

<img src="img/architecture.png" style="width:700px;height:400;">
<caption><center> <u> **Figure 2** </u>: **Encoding architecture for YOLO**<br> </center></caption>

如果一个目标的中心点若在特定网格内，那么这个网格需要检测出对应的目标。

考虑到我们使用了5个archor boxes，那么19 × 19的网格，每一个都包含了5个archor boxes的信息。每个anchor boxes由他们的宽和高来定义。

为了方便，我们将 (19, 19, 5, 85) 编码的最后两个维度打平，所以深度圣经网络的输出将会是 (19, 19, 425)。

<img src="img/flatten.png" style="width:700px;height:400;">
<caption><center> <u> **Figure 3** </u>: **Flattening the last two last dimensions**<br> </center></caption>

这样，对于（每个网格的）每个box，，我们需要计算元素级别的乘积，抽取出每个box包含每个分类的概率值。

<img src="img/probability_extraction.png" style="width:700px;height:400;">
<caption><center> <u> **Figure 4** </u>: **Find the class detected by each box**<br> </center></caption>

下面是对YOLO预测算法进行可视化的一种方法：
- 对19×19的每个网格，找到其最大的概率值（对5个archor boxes，每个都对80的分类，取最大值）
- 对每个网格，根据最可能的分类，进行上色

可视化的效果如下: 

<img src="img/proba_map.png" style="width:300px;height:300;">
<caption><center> <u> **Figure 5** </u>: Each of the 19x19 grid cells colored according to which class has the largest predicted probability in that cell.<br> </center></caption>

这个可视化方法并不是YOLO算法进行预测的核心组成部分，但它可以很好地展示算法的中间结果。

另一种YOLO输出可视化的办法，是直接在原图上绘制产出的包围盒，可视化的效果如下：

<img src="img/anchor_map.png" style="width:200px;height:200;">
<caption><center> <u> **Figure 6** </u>: Each cell gives you 5 boxes. In total, the model predicts: 19x19x5 = 1805 boxes just by looking once at the image (one forward pass through the network)! Different colors denote different classes. <br> </center></caption>

在上图中，我们仅仅绘制了概率较高的包围盒，但包围盒的数量仍然过多。我们需要将算法的输出结果，过滤为更小的目标集合。为了达到这个目标，我们需要使用非极大值抑制。具体来说，从模型产出的19×19×5个包围盒开始，我们需要：
- 去除概率较低的包围盒（说明这个包围盒检测到分类的置信度较低）
- 对于相互重合的包围盒，仅选取一个最可信的